# Steps:

1. Download data
    * create validation data set
    * create sample set
2. setup VGG model
3. load train and validation data sets
4. train on batches
5. predict

In [1]:
%matplotlib inline

## 1. Download data

Run:

password=[...] sh download.sh

In [1]:
import os 
dir_path = os.getcwd()
dir_path

'/home/ubuntu/fast-ai/deeplearning1/nbs/lesson1'

In [2]:
path=dir_path + "/data/"

## 2. Setup VGG

In [3]:
from vgg16 import Vgg16

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [4]:
from keras.preprocessing import image

In [5]:
vgg = Vgg16()

## 3. Load data

In [7]:
batch_size=64
nb_epochs = 10

In [8]:
gen = image.ImageDataGenerator(rotation_range=5,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3,horizontal_flip=True)
batches = vgg.get_batches(path+"train", batch_size=batch_size, gen=gen)
val_batches = vgg.get_batches(path+"valid", batch_size=batch_size*2,gen=gen)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## 4. Train

In [9]:
vgg.finetune(batches)

In [ ]:
# vgg.fit(batches, val_batches, nb_epoch=2)
latest_weights_fname = None
for e in range(nb_epochs):
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_fname = "epoch-%d.h5" % e
    vgg.model.save_weights(latest_weights_fname)
    vgg.model.optimizer.lr /= e

Epoch 1/1
23000/23000 [==============================] - 623s - loss: 0.1719 - acc: 0.9541 - val_loss: 0.1019 - val_acc: 0.9685

## 5. Test

In [ ]:
batches, preds = vgg.test(path+"test", batch_size=batch_size*2)

Found 12500 images belonging to 1 classes.


In [ ]:
filenames = batches.filenames

Here's the format Kaggle requires for new submissions:
```
imageId,isDog
1242, .3984
3947, .1000
4539, .9082
2345, .0000
```

Kaggle wants the imageId followed by the probability of the image being a dog. Kaggle uses a metric called [Log Loss](http://wiki.fast.ai/index.php/Log_Loss) to evaluate your submission.

In [ ]:
is_dog = preds[:, 1]

In [ ]:
import numpy as np
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
sub = np.stack([ids, is_dog], axis=1)
sub[:5]

In [ ]:
%cd data

In [ ]:
np.savetxt("subm.csv", sub, fmt="%d,%.5f", header="id,label", comments="")

In [ ]:
from IPython.display import FileLink
%cd ..
FileLink("data/subm.csv")